In [ ]:
!nvidia-smi

In [ ]:
#Поставим бибилотеку для аугментации
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
# Поставим модуль с предобученными сетями efficientnet
!pip install -U efficientnet

In [ ]:
import albumentations
from ImageDataAugmentor.image_data_augmentor import *
import efficientnet.tfkeras as efn

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow.keras.optimizers as O
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, StratifiedKFold

from statistics import mean

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

# 0.93408 - результат Base line

# Первое изменение. 
1. Увеличим размер картинки  
2. Увеличим количесатво эпох обучения

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 10  # эпох на обучение
BATCH_SIZE           = 32 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.2 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE_X           = 250 
IMG_SIZE_Y           = 250

IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE_X, IMG_SIZE_Y, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория

RANDOM_SEED = 33

# EDA (Разведывательный анализ данных)

Считаем csv тренировочного ДФ

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(f'{PATH}/train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры, чтобы понимать, как их лучше обрабатывать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100389.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

In [ ]:
# Посмотрим какого размера картинkи
image_size_x = 0
image_size_y = 0

image_size_x_list = []
image_size_y_list = []

for i in range(len(train_df)):
    #image_size_x += PIL.Image.open(PATH+'/train/'+str(train_df['Category'][i])+'/'+train_df['Id'][i]).size[0]
    image_size_x_list.append(PIL.Image.open(PATH+'/train/'+str(train_df['Category'][i])+'/'+train_df['Id'][i]).size[0])
    #image_size_y += PIL.Image.open(PATH+'/train/'+str(train_df['Category'][i])+'/'+train_df['Id'][i]).size[1]
    image_size_y_list.append(PIL.Image.open(PATH+'/train/'+str(train_df['Category'][i])+'/'+train_df['Id'][i]).size[1]) 
print('Средняя длина картинки', mean(image_size_x_list))
print('Max длина картинки', max(image_size_x_list))
print('Min длина картинки', min(image_size_x_list))
print('Средняя ширина картинки', mean(image_size_y_list))
print('Max ширина картинки', max(image_size_y_list))
print('Min ширина картинки', min(image_size_y_list))

# Подготовка данных

# Второе изменение - используем новую аугментацию

## АУГМЕНТАЦИЯ ДАННЫХ  
Добавим парамтеров для аугментации  
Изменим размер картинок на максимальный

In [ ]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    albumentations.FancyPCA (alpha=0.1, always_apply=False, p=0.5),
    albumentations.Transpose(p=0.5),
    albumentations.ShiftScaleRotate(p=0.5),
    albumentations.ElasticTransform(),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.1),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.CenterCrop(height=200, width=224),
    ],p=0.5),    
    albumentations.GaussianBlur(p=0.1),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.Resize(IMG_SIZE_X, IMG_SIZE_Y)
])

In [ ]:
train_datagen = ImageDataAugmentor(
        rescale = 1/255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )

test_datagen = ImageDataGenerator(rescale = 1/255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE_X, IMG_SIZE_Y),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    #interpolation = 'wrap', #пригодится если картинка будем меньше указанного размера  Но почему-то не работает
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE_X, IMG_SIZE_Y),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    #interpolation = 'wrap', #пригодится если картинка будем меньше указанного размера    Но почему-то не работает
    subset='validation') # set as validation data

test_sub_generator = train_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    #interpolation = 'wrap',  Но почему-то не работает
    target_size=(IMG_SIZE_X, IMG_SIZE_Y),
    batch_size=BATCH_SIZE,)

# Изменение три - улучшение модели

### 3.1 Возьмём модель c более высокой точостью среди тех, у которых мало параметров для imagenet в качестве базовой

In [ ]:
base_model = efn.EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMG_SIZE_X,IMG_SIZE_Y,3))

### 3.2 Подкрутим голову

In [ ]:
K.clear_session()

model=M.Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(L.GlobalAveragePooling2D())
model.add(L.Flatten()) 
model.add(L.Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(L.Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(L.Dense(CLASS_NUM, activation='softmax'))

# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
model.summary()

### 3.3 Усовершенствуем callbacks  
Здусь же поупражняемся со скоростью обучения

In [ ]:
### Данная функция будет меняться перед каждым обучением для уменьшения LR
#def make_callbacks():
#    
#    callback_early_stopping = EarlyStopping(monitor='accuracy',patience=10, verbose=1)
#    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5,min_lr=1e-10,patience=0,verbose=1)
#    callback_learing_rate = LearningRateScheduler(lambda x: 1e-3 * 0.8 ** x), # Формула для уменьшения LR
#    callback_checkpoint = ModelCheckpoint(f'{model}_best.hdf5' , monitor = ['val_acc'] , verbose = 1  , mode = 'max') 
#
#    return [callback_early_stopping,
#            callback_reduce_lr,
#            callback_learing_rate,
#            callback_checkpoint,
#            ]
#
#callbacks_list = make_callbacks()

### Пофаинтюним, но немного. 

Сначала обучим голову

In [ ]:
# Зададим LR
LR = 0.001

def make_callbacks():
    
    callback_early_stopping = EarlyStopping(monitor='accuracy',patience=4, verbose=1)
    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5,min_lr=1e-10,patience=0,verbose=1)
    callback_learing_rate = LearningRateScheduler(lambda x: 0.001 * 0.95 ** x),
    callback_checkpoint = ModelCheckpoint('model_1_best.hdf5' , monitor = ['val_acc'] , verbose = 1  , mode = 'max') 

    return [callback_early_stopping,
            callback_reduce_lr,
            callback_learing_rate,
            callback_checkpoint,
            ]

callbacks_list = make_callbacks()

In [ ]:
base_model.trainable = False

print('Выполняется обучение', len(base_model.trainable_variables), 'слоев базовой модели')

In [ ]:
train_generator.reset()
history = model.fit(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = test_generator, 
    validation_steps = len(test_generator),
    epochs = EPOCHS,
    callbacks = callbacks_list)

In [ ]:
print('Сохраняю модель')
model.save('model_1_last_finetuning.hdf5')

In [ ]:
#print('Загружаю модель')    
#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
#model.load_weights('../input/model-2-best/model_2_best.hdf5') # я скачивал результаты обучения на ПК и после импортивал в ноутбук
#scores = model.evaluate_generator(test_generator, verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Зададим LR
LR = 0.0001

def make_callbacks():
    
    callback_early_stopping = EarlyStopping(monitor='accuracy',patience=4, verbose=1)
    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5,min_lr=1e-10,patience=0,verbose=1)
    callback_learing_rate = LearningRateScheduler(lambda x: 0.0001 * 0.95 ** x),
    callback_checkpoint = ModelCheckpoint('model_3_best.hdf5' , monitor = ['val_acc'] , verbose = 1  , mode = 'max') 

    return [callback_early_stopping,
            callback_reduce_lr,
            callback_learing_rate,
            callback_checkpoint,
            ]

callbacks_list = make_callbacks()

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:305]:
    layer.trainable =  False
print('Выполняется обучение', len(base_model.trainable_variables), 'слоев базовой модели')

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = test_generator, 
    validation_steps = len(test_generator),
    epochs = 10,
    callbacks = callbacks_list)

In [ ]:
print('Сохраняю модель')
model.save('model_2_last_finetuning.hdf5')

In [ ]:
#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
print('Загружаю модель')    
#model.load_weights('../input/best-model-3/model_3_best (2).hdf5') # я скачивал результаты обучения на ПК и после импортивал в ноутбук, и да, тут есть путаница в названиях
#scores = model.evaluate_generator(test_generator, verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Отредактируем стартовый LR
LR = 0.00005

def make_callbacks():
    
    callback_early_stopping = EarlyStopping(monitor='accuracy',patience=4, verbose=1)
    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5,min_lr=1e-10,patience=0,verbose=1)
    callback_learing_rate = LearningRateScheduler(lambda x: LR * 0.9 ** x),
    callback_checkpoint = ModelCheckpoint('model_3_best.hdf5' , monitor = ['val_acc'] , verbose = 1  , mode = 'max') 

    return [callback_early_stopping,
            callback_reduce_lr,
            callback_learing_rate,
            callback_checkpoint,
            ]

callbacks_list = make_callbacks()

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:105]:
    layer.trainable =  False
print('Выполняется обучение', len(base_model.trainable_variables), 'слоев базовой модели')

In [ ]:
train_generator.reset()
history = model.fit(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = test_generator, 
    validation_steps = len(test_generator),
    epochs = 10, # хочется 15, но придется потом ждать неделю...
    callbacks = callbacks_list)

In [ ]:
print('Сохраняю модель')
model.save('model_3_last_finetuning.hdf5')
print('Загружаю модель')    
model.load_weights('model_3_best.hdf5')
#scores = model.evaluate_generator(test_generator, verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Отредактируем стартовый LR
LR = 0.000005

def make_callbacks():
    
    callback_early_stopping = EarlyStopping(monitor='accuracy',patience=4, verbose=1)
    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5,min_lr=1e-10,patience=0,verbose=1)
    callback_learing_rate = LearningRateScheduler(lambda x: LR * 0.75 ** x),
    callback_checkpoint = ModelCheckpoint('model_4_best.hdf5' , monitor = ['val_acc'] , verbose = 1  , mode = 'max') 

    return [callback_early_stopping,
            callback_reduce_lr,
            callback_learing_rate,
            callback_checkpoint,
            ]

callbacks_list = make_callbacks()

In [ ]:
base_model.trainable = True

print('Выполняется обучение', len(base_model.trainable_variables), 'слоев базовой модели')

In [ ]:
train_generator.reset()
history = model.fit(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = test_generator, 
    validation_steps = len(test_generator),
    epochs = 8,
    callbacks = callbacks_list)

In [ ]:
print('Сохраняю модель')
model.save('model_4_last_finetuning.hdf5')
#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
#print('Загружаю модель')    
#model.load_weights('model_4_best.hdf5')
#scores = model.evaluate_generator(test_generator, verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Построим графики последнего обучения
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

### Улучшение 5  
Усовершенствуем предсказание  
Будем делать предсказания на аугментированных данных, после чего брать наиболее часто повторяющиеся.

In [ ]:
## загрузим лучшую модель
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
print('Загружаю модель')    
model.load_weights('../input/my-best-model/model_4_best (8).hdf5')
print('Модель загружен')

In [ ]:
test_sub_generator.reset()

tta_steps = 10 # берем среднее из 10 предсказаний
predictions = []

for i in range(tta_steps):
    preds = model.predict(test_sub_generator, steps=len(test_sub_generator), verbose=1)
    predictions.append(preds)
    test_sub_generator.reset()

pred = np.mean(predictions, axis=0)

# Сделаем предсказание

In [ ]:
#test_sub_generator.reset()
#predictions = model.predict(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
#predictions = np.argmax(predictions, axis=-1) #multiple categories
#label_map = (train_generator.class_indices)
#label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
#predictions = [label_map[k] for k in predictions]

In [ ]:
#filenames_with_dir=test_sub_generator.filenames
#submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
#submission['Id'] = submission['Id'].replace('test_upload/','')
#submission.to_csv('submission.csv', index=False)
#print('Save submit')

In [ ]:
predictions = np.argmax(pred, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission_TTA.csv', index=False)
print('Save submit')

# Что можно было бы сделать лучше?  

1. Я бы могу собрать с авто.ру огромный датасет, не менее чем еще на 100 000 картинок, но это бы заняло у меня 2 - 3 дня, которых у меня не было. Бызовый код взял бы из позапрошлого проекта, но вместо всех параметров тянул бы только картинки и названия марки.
2. Так же я плохо разобрался с LR. Я так и не понял имеет-ли смысл задавать "расписание" для LR при компиляции модели в оптимайзере, если оно задается в функции callbacks. Я думаю, что проработка этого вопроса помогла бы мне улучшить результат.